# Creating a Response Dataset using Argilla Distilabel and Hugging Face Spaces

This notebook walks through the process of collecting translated prompts from one or many completed or in progress [MPEP datasets](https://huggingface.co/datasets?sort=trending&search=mpep), querying a configurable jury of open source LLM models, collecting their responses, synthesizing data using [Distilabel](https://distilabel.argilla.io/latest/)

In [1]:
# Import required libraries

from datasets import load_dataset
from dotenv import load_dotenv
from huggingface_hub import HfApi, HfFolder, login
# import markdown
import os
import pandas as pd
import json
# import requests
# import subprocess
# import transformers

# Need to pip install the below

from distilabel.llms import InferenceEndpointsLLM
from distilabel.pipeline import Pipeline
from distilabel.steps import LoadDataFromHub
from distilabel.steps.tasks import TextGeneration

/opt/anaconda3/envs/hf_n/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# HF Login

load_dotenv()

hf_token = os.getenv('HF_TOKEN')
HfFolder.save_token(hf_token)

if hf_token is None:
    raise ValueError("HF_TOKEN environment variable not set.")

login(token=hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/kenneth.hamilton/.cache/huggingface/token
Login successful


In [3]:
# Collect and print MPEP datasets
api = HfApi()

def search_datasets_by_label(label):
    datasets = api.list_datasets(search=label)
    return list(datasets)  # Convert generator to list

mpep_datasets = search_datasets_by_label('MPEP')

# Print the datasets found
print(f"Found {len(mpep_datasets)} MPEP datasets")

Found 11 MPEP datasets


In [5]:
# Directories to save responses and cache
os.makedirs('./responses', exist_ok=True)
os.makedirs('./cache', exist_ok=True)

# Load existing cache files
processed_external_ids = {}
for cache_file in os.listdir('./cache'):
    if cache_file.endswith('.txt'):
        language = cache_file.replace('.txt', '')
        with open(f'./cache/{cache_file}', 'r') as f:
            processed_external_ids[language] = set(f.read().splitlines())

# Load models configuration
with open('models.json', 'r') as f:
    models_config = json.load(f)

In [6]:
for dataset in mpep_datasets:
    print(f"Processing dataset: {dataset.id}")

    ds = load_dataset(dataset.id)
    if len(ds['train']) < 500:
        print(f"Skipping dataset {dataset.id} as it has less than 500 prompts.")
        continue

    target_entries = ds['train']['target'][:5]
    external_ids = ds['train']['external_id'][:5]

    language = dataset.id.split('_')[-1].lower()

    if language not in processed_external_ids:
        processed_external_ids[language] = set()

    with open(f'./responses/{language}.md', 'w') as file:
        for model_config in models_config['models']:
            model_endpoint = model_config['endpoint_name']
            generation_params = model_config['params']

            print(f"Using model: {model_endpoint} with params: {generation_params}")

            with Pipeline(name='mpe_pipeline') as pipeline:
                load_dataset_step = LoadDataFromHub(
                    name='load_dataset',
                    repo_id=dataset.id,
                    split='train',
                    input_mappings={},
                    output_mappings={'target': 'instruction'},
                    batch_size=5
                )

                text_generation_step = TextGeneration(
                    name='text_generation',
                    llm=InferenceEndpointsLLM(
                        model_id=None,
                        endpoint_name=model_endpoint,
                        generation_kwargs=generation_params
                    ),
                    input_mappings={'instruction': 'instruction'},
                    output_mappings={'generation': 'response'},
                    input_batch_size=5
                )

                load_dataset_step >> text_generation_step

            distiset = pipeline.run()

            for entry in target_entries:
                prompt = entry['source']
                response_text = pipeline.execute({'prompt': prompt})['response']
                file.write(f"### Prompt:\n{prompt}\n\n### Response:\n{response_text}\n\n")
                file.flush()

    with open(f'./cache/{language}.txt', 'w') as f:
        f.write('\n'.join(processed_external_ids[language]))

Processing dataset: DIBT/MPEP_GERMAN
Skipping dataset DIBT/MPEP_GERMAN as it has less than 500 prompts.
Processing dataset: DIBT/MPEP_DUTCH
Using model: google/gemma-2-27b-it with params: {'temperature': 0.7, 'max_new_tokens': 512}


[07/19/24 10:49:01] INFO     ['distilabel.pipeline'] 📝 Pipeline data will be written to                ]8;id=383687;file:///opt/anaconda3/envs/hf_n/lib/python3.8/site-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=402510;file:///opt/anaconda3/envs/hf_n/lib/python3.8/site-packages/distilabel/pipeline/base.py#611\611]8;;\
                             '/Users/kenneth.hamilton/.cache/distilabel/pipelines/mpe_pipeline/769feee6            
                             abcbb1d8632fce5fe8a65a499ed65e4c/data'                                                

                    INFO     ['distilabel.pipeline'] ⏳ Waiting for all the steps to load...            ]8;id=205367;file:///opt/anaconda3/envs/hf_n/lib/python3.8/site-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=971667;file:///opt/anaconda3/envs/hf_n/lib/python3.8/site-packages/distilabel/pipeline/base.py#703\703]8;;\

[07/19/24 10:49:04] ERROR    ['distilabel.pipeline'] ❌ Failed with an unhandled exception:            ]8;id=774475;file:///opt/anaconda3/envs/hf_n/lib/python3.8/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=261343;file:///opt/anaconda3/envs/hf_n/lib/python3.8/site-packages/distilabel/pipeline/local.py#203\203]8;;\
                             format_exception() missing 2 required positional arguments: 'value' and               
                             'tb'                                                                                  

                    INFO     ['distilabel.pipeline'] 🛑 Stopping pipeline. Waiting for steps to finish ]8;id=771035;file:///opt/anaconda3/envs/hf_n/lib/python3.8/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=850611;file:///opt/anaconda3/envs/hf_n/lib/python3.8/site-packages/distilabel/pipeline/local.py#273\273]8;;\
                             processing batches...                                                                 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:48                                                                                   │
│                                                                                                  │
│   45 │   │   │   │                                                                               │
│   46 │   │   │   │   load_dataset_step >> text_generation_step                                   │
│   47 │   │   │                                                                                   │
│ ❱ 48 │   │   │   distiset = pipeline.run()                                                       │
│   49 │   │   │                                                                                   │
│   50 │   │   │   for entry in target_entries:                                                    │
│   51 │   │   │   │   prompt = entry['source']                                                    │
│                                                                                                  │
│ ╭─────────────────────────────────────────── locals ───────────────────────────────────────────╮ │
│ │                      api = <huggingface_hub.hf_api.HfApi object at 0x16c333e50>              │ │
│ │               cache_file = 'dutch.txt'                                                       │ │
│ │                  dataset = DatasetInfo(                                                      │ │
│ │                            │   id='DIBT/MPEP_DUTCH',                                         │ │
│ │                            │   author='DIBT',                                                │ │
│ │                            │   sha='5eeae30facb4c319282b4b4d1a45153e2866d188',               │ │
│ │                            │   created_at=datetime.datetime(2024, 4, 4, 10, 15, 40,          │ │
│ │                            tzinfo=datetime.timezone.utc),                                    │ │
│ │                            │   last_modified=datetime.datetime(2024, 4, 29, 16, 4, 18,       │ │
│ │                            tzinfo=datetime.timezone.utc),                                    │ │
│ │                            │   private=False,                                                │ │
│ │                            │   gated=False,                                                  │ │
│ │                            │   disabled=False,                                               │ │
│ │                            │   downloads=68,                                                 │ │
│ │                            │   likes=2,                                                      │ │
│ │                            │   paperswithcode_id=None,                                       │ │
│ │                            │   tags=[                                                        │ │
│ │                            │   │   'size_categories:n<1K',                                   │ │
│ │                            │   │   'format:parquet',                                         │ │
│ │                            │   │   'modality:text',                                          │ │
│ │                            │   │   'library:datasets',                                       │ │
│ │                            │   │   'library:pandas',                                         │ │
│ │                            │   │   'library:mlcroissant',                                    │ │
│ │                            │   │   'library:argilla',                                        │ │
│ │                            │   │   'region:us',                                              │ │
│ │                            │   │   'rlfh',                                                   │ │
│ │                            │   │   'argilla',                                                │ │
│ │                            │   │   ... +1                

In [ ]:
import os

from distilabel.llms import InferenceEndpointsLLM
from distilabel.pipeline import Pipeline
from distilabel.steps.tasks import TextGeneration

endpoint_name = "aws-notus-7b-v1-4052" or os.getenv("HF_INFERENCE_ENDPOINT_NAME")
endpoint_namespace = "argilla" or os.getenv("HF_NAMESPACE")

pipe_generation = Pipeline(
    generator=InferenceEndpointsLLM(
        endpoint_name_or_model_id=endpoint_name,  # The name given of the deployed model
        endpoint_namespace=endpoint_namespace,  # This usually corresponds to the organization, in this case "argilla"
        token=os.getenv("HF_TOKEN"),  # hf_...
        task=TextGeneration(),
        max_new_tokens=512,
        do_sample=True,
        prompt_format="notus",
    ),
)